In [1]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [2]:
import keras
print(keras.__version__)

2.7.0


TF-IDF 생성 후 심층 신경망을 이용한 이메일 분류 


In [3]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [5]:
x_train = newsgroups_train.data
x_test = newsgroups_test.data

In [6]:
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [7]:
print ("20개 카테고리 전체 목록:")
print (newsgroups_train.target_names)
print ("\n")
print ("샘플 이메일:")
print (x_train[0])
print ("샘플 타깃 카테고리:")
print (y_train[0])
print (newsgroups_train.target_names[y_train[0]])

20개 카테고리 전체 목록:
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


샘플 이메일:
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, o

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [21]:
def preprocessing(text):
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())

    tokens = [word for sent in nltk.sent_tokenize(text2) for word in
              nltk.word_tokenize(sent)]
    
    tokens = [word.lower() for word in tokens]
    
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    
    tokens = [word for word in tokens if len(word)>=3]
    
    stemmer = PorterStemmer()
    try:
        tokens = [stemmer.stem(word) for word in tokens]

    except:
        tokens = tokens
        
    tagged_corpus = pos_tag(tokens)    
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']

    lemmatizer = WordNetLemmatizer()

    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    
    pre_proc_text =  " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])             

    return pre_proc_text

In [22]:
x_train_preprocessed  = []
for i in x_train:
	x_train_preprocessed .append(preprocessing(i))

x_test_preprocessed = []
for i in x_test:
	x_test_preprocessed.append(preprocessing(i))

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2),  stop_words='english', 
                             max_features= 10000,strip_accents='unicode',  norm='l2')

x_train_2 = vectorizer.fit_transform(x_train_preprocessed).todense()
x_test_2 = vectorizer.transform(x_test_preprocessed).todense()

In [30]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils

In [32]:
np.random.seed(1337)  # 랜덤 시드 고정
nb_classes = 20
batch_size = 64
nb_epochs = 20

Y_train = np_utils.to_categorical(y_train, nb_classes)

In [33]:
model = Sequential() # 순차 모델 정의

model.add(Dense(1000,input_shape= (10000,))) # 출력될 뉴런의 수, 입력차원
model.add(Activation('relu')) # 노드를 줄여가는 비율
model.add(Dropout(0.5))

model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax')) # 출력층

model.compile(loss='categorical_crossentropy', optimizer='adam') # 옵티마이저
print (model.summary()) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              10001000  
                                                                 
 activation (Activation)     (None, 1000)              0         
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 activation_1 (Activation)   (None, 500)               0         
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                2

In [34]:
# 모델 학습 과정
model.fit(x_train_2, Y_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

Epoch 1/20
177/177 [==============================] - 17s 93ms/step - loss: 2.0022
Epoch 2/20
177/177 [==============================] - 16s 93ms/step - loss: 0.6157
Epoch 3/20
177/177 [==============================] - 16s 91ms/step - loss: 0.3063
Epoch 4/20
177/177 [==============================] - 16s 90ms/step - loss: 0.1874
Epoch 5/20
177/177 [==============================] - 16s 89ms/step - loss: 0.1119
Epoch 6/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0813
Epoch 7/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0661
Epoch 8/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0605
Epoch 9/20
177/177 [==============================] - 16s 88ms/step - loss: 0.0484
Epoch 10/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0404
Epoch 11/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0399
Epoch 12/20
177/177 [==============================] - 16s 89ms/step - loss: 0.0422
E

In [37]:
y_train_predclass = np.argmax(model.predict(x_train_2),axis = 1)
y_test_predclass =np.argmax(model.predict(x_test_2),axis = 1)

In [38]:
from sklearn.metrics import accuracy_score,classification_report

print ("\n\nDeep Neural Network - Train accuracy:"),(round(accuracy_score(y_train,y_train_predclass),3))
print ("\nDeep Neural Network - Test accuracy:"),(round(accuracy_score(y_test,y_test_predclass),3))

print ("\nDeep Neural Network - Train Classification Report")
print (classification_report(y_train,y_train_predclass))

print ("\nDeep Neural Network - Test Classification Report")
print (classification_report(y_test,y_test_predclass))



Deep Neural Network - Train accuracy:

Deep Neural Network - Test accuracy:

Deep Neural Network - Train Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       480
           1       1.00      1.00      1.00       584
           2       1.00      1.00      1.00       591
           3       1.00      1.00      1.00       590
           4       1.00      1.00      1.00       578
           5       1.00      1.00      1.00       593
           6       1.00      1.00      1.00       585
           7       1.00      1.00      1.00       594
           8       1.00      1.00      1.00       598
           9       1.00      1.00      1.00       597
          10       1.00      1.00      1.00       600
          11       1.00      1.00      1.00       595
          12       1.00      1.00      1.00       591
          13       1.00      1.00      1.00       594
          14       1.00      1.00      1.00       593
      